# 전력 수요량 예측 프로젝트

- **조 이름**: 스마트에너지예측조
- **참여자**: 김도영, 홍길동, 이지은
- **주제**: 한국전력거래소의 시간별 전력 수요량 데이터를 활용한 시계열 예측
- **예측 목표**: 향후 24시간의 전력 수요량(MWh) 예측
- **사용 모델**:
  - ARIMA: 전통적 통계 기반 시계열 모델
  - N-BEATS: MLP 기반의 딥러닝 예측 모델
  - TCN: CNN 기반 시계열 예측 딥러닝 모델
  - TiDE: Transformer 계열의 시계열 특화 딥러닝 모델
  - 앙상블: 위 모델들의 단순 평균 앙상블 모델


In [2]:
# 📦 필요한 라이브러리 설치
!pip install darts[torch] --quiet
!pip install matplotlib pandas --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.6/200.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.1/823.1 kB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.4/354.4 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:

# -- 1. 라이브러리 설치 및 불러오기 --
# Colab 환경에서 최초 실행 시 아래 주석 해제 후 설치 필요
# !pip install darts[torch]

import pandas as pd
import matplotlib.pyplot as plt
from darts import TimeSeries
from darts.models import ARIMA, NBEATSModel, TCNModel, TiDEModel, RegressionEnsembleModel
from darts.metrics import mae, rmse
from darts.dataprocessing.transformers import Scaler
from darts.utils.missing_values import fill_missing_values
from darts.utils.utils import ModelMode

# -- 2. 데이터 불러오기 및 전처리 --
# 전력수요 데이터를 wide → long 형태로 변환하고, '24시'는 다음날 0시로 이동
df = pd.read_csv('한국전력거래소_시간별 전국 전력수요량_20241231.csv', encoding='cp949')

# melt: '1시'~'24시' 열을 시간/수요량으로 재구성
df_long = df.melt(id_vars='날짜', var_name='시간', value_name='수요량(MWh)')
df_long['시간_num'] = df_long['시간'].str.replace('시', '').astype(int)

# 시간정보 합치기 (24시는 00시로 간주하고 다음날로 이동)
df_long['일시'] = pd.to_datetime(df_long['날짜']) + pd.to_timedelta(df_long['시간_num'] % 24, unit='h')
df_long.loc[df_long['시간_num'] == 24, '일시'] += pd.Timedelta(days=1)

# 필요한 컬럼 정리
df_long = df_long[['일시', '수요량(MWh)']].sort_values('일시')

In [6]:
# -- 3. TimeSeries 객체 생성 및 결측치 처리 --
series = TimeSeries.from_dataframe(df_long, time_col='일시', value_cols='수요량(MWh)')
series = fill_missing_values(series)



In [7]:
# -- 4. 학습/테스트 데이터 분리 --
train, val = series.split_before(0.8)



In [8]:
# -- 5. 스케일링 (딥러닝 모델은 꼭 필요) --
scaler = Scaler()
train_scaled = scaler.fit_transform(train)
val_scaled = scaler.transform(val)



In [9]:
# -- 6. ARIMA 모델 (기본 통계 기반 예측 모델) --
arima = ARIMA()
arima.fit(train)
arima_pred = arima.predict(len(val))
arima_mae = mae(val, arima_pred)
arima_rmse = rmse(val, arima_pred)

In [ ]:
# -- 7. N-BEATS 모델 (MLP 기반 딥러닝 시계열 예측) --
nbeats = NBEATSModel(
    input_chunk_length=168,
    output_chunk_length=24,
    n_epochs=30,
    batch_size=32,
    random_state=42,
    model_name="nbeats_power",
    force_reset=True
)
nbeats.fit(train_scaled, verbose=True)
nbeats_pred = scaler.inverse_transform(nbeats.predict(len(val_scaled)))
nbeats_mae = mae(val, nbeats_pred)
nbeats_rmse = rmse(val, nbeats_pred)




INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | stacks          | ModuleList       | 7.3 M  | train
-------------------------------------------------------------
7.3 M     Trainable params
2.3 K     Non-trainable params
7.3 M     Total params
29.323    Total estimated model params size (MB)
396       Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

In [ ]:
# -- 8. TCN 모델 (Conv 기반 딥러닝 시계열 예측) --
tcn = TCNModel(
    input_chunk_length=168,
    output_chunk_length=24,
    kernel_size=3,
    num_filters=16,
    n_epochs=30,
    batch_size=32,
    random_state=42,
    model_name="tcn_power",
    force_reset=True
)
tcn.fit(train_scaled, verbose=True)
tcn_pred = scaler.inverse_transform(tcn.predict(len(val_scaled)))
tcn_mae = mae(val, tcn_pred)
tcn_rmse = rmse(val, tcn_pred)





In [ ]:
# -- 9. TiDE 모델 (Transformer 기반 시계열 예측 최신 모델) --
tide = TiDEModel(
    input_chunk_length=168,
    output_chunk_length=24,
    n_epochs=30,
    batch_size=32,
    random_state=42,
    model_name="tide_power",
    force_reset=True
)
tide.fit(train_scaled, verbose=True)
tide_pred = scaler.inverse_transform(tide.predict(len(val_scaled)))
tide_mae = mae(val, tide_pred)
tide_rmse = rmse(val, tide_pred)



In [11]:
# -- 10. 앙상블 모델 (단순 평균 기반) --
ensemble = RegressionEnsembleModel(
    models=[arima, nbeats, tcn, tide]
)

ensemble.fit(
    series=train_scaled,
    past_covariates=None,  # 과거 변수 없을 경우 None
    forecast_horizon=24,
    regression_train_n_points=168
)

ensemble_pred = scaler.inverse_transform(ensemble.predict(len(val_scaled)))
ensemble_mae = mae(val, ensemble_pred)
ensemble_rmse = rmse(val, ensemble_pred)


TypeError: RegressionEnsembleModel.__init__() got an unexpected keyword argument 'models'

In [ ]:
# -- 11. 시각화: 예측 결과 비교 --
plt.figure(figsize=(15,6))
val.plot(label='실제값', lw=2)
arima_pred.plot(label='ARIMA', lw=1)
nbeats_pred.plot(label='N-BEATS', lw=1)
tcn_pred.plot(label='TCN', lw=1)
tide_pred.plot(label='TiDE', lw=1)
ensemble_pred.plot(label='앙상블', lw=2, color='black')
plt.legend()
plt.title('모델별 전력수요 예측 비교')
plt.show()


In [ ]:
# -- 12. 성능 비교 테이블 출력 --
results = pd.DataFrame({
    '모델': ['ARIMA', 'N-BEATS', 'TCN', 'TiDE', '앙상블'],
    'MAE': [arima_mae, nbeats_mae, tcn_mae, tide_mae, ensemble_mae],
    'RMSE': [arima_rmse, nbeats_rmse, tcn_rmse, tide_rmse, ensemble_rmse]
})
print(results)